# Linear Regression

<a id="introduce-the-bikeshare-dataset"></a>
## Introduce the Capital Bikeshare Data Set
---

- This dataset contains the hourly and daily count of rental bikes between years 2011 and 2012 in Capital bikeshare system with the corresponding weather and seasonal informat (http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset)
- The goal is to predict how many bikes will be rented depending on the weather and the day.
- Here are the list of features that we are going to use:
 * datetime - hourly date + timestamp  
 * season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
 * holiday - whether the day is considered a holiday
 * workingday - whether the day is neither a weekend nor holiday
 * weather:
        1. Clear, Few clouds, Partly cloudy, Partly cloudy 
        2. Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
        3. Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
        4. Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
 * temp - temperature in Celsius
 * atemp - "feels like" temperature in Celsius
 * humidity - relative humidity
 * windspeed - wind speed
 * casual - number of non-registered user rentals initiated
 * registered - number of registered user rentals initiated
 * count - number of total rentals

## Exploratory Data Analysis

When working with real dataset, there are a lot of processes that we need to go through in order to correctly apply an ML model to solve the problems. This deserves a separate chapter to describe the entire process. The below example sets out to demonstrate some key processes that we all have to go through in handling any data project. 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14
plt.style.use("fivethirtyeight")

In [ ]:
# Read the data and set the datetime as the index.
bikes = pd.read_csv("data/bikeshare.csv")


In [ ]:
# Check out the types of data in each of the column. This gives us an idea of which operations we should perform on each column and whether 
# we need to convert any column to new data structure.


In [ ]:
# datetime is better represented as a datetime datatype


Create a new column called `hour` out of `datetime` column

In [ ]:
# Check for missing data. This data has been cleaned so it does not have missing values


Here are some of the initial findings:
- Each row or observation contains the values of each feature and the number of bikes rented during an hour, which is the target we want to predict
- There are 9 features in this dataset: **datetime, season, holiday, workingday, weather, temp, atemp, humidity, windspeed**
- The target variable is: **count**, which is broken down into **casual** and **registered**

Rename the column `count` to `total_rentals` to avoid confusion with keywords conflict

In [ ]:
# Use the .rename() method to rename count to total_rentals


Summer and Fall seems to have rental counts a bit higher than in Spring and Winter

There are very few rentals when the weather is not nice (Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog)

There is a clear difference between holiday and non-holiday rents. People rent bikes to work or school on non-holiday periods

We can do pairwise visualisation on the features to detect whether there is any interesting correlations between some of the features.
If a pair of features is highly correlated, we need to remove one of them from the input to the model to avoid colinearity.

In [ ]:
# Use heatmap as a different method to observe the correlation
# Change the colour map from -1.0 to 1.0
plt.figure(figsize = (16,9)) # change the size of heatmap
sns.heatmap(bikes.corr(), ## this is correlation between features, should not normalise them to -1 to 1, or mess up the values
            cmap='coolwarm', 
            annot=True, 
            robust=True, 
            fmt=".2f",
            annot_kws={'size':12}
           )
plt.savefig("images/eda_05.png")

In [ ]:
# Try out some simple visualisation to see the relationship between some variables and the target
bikes.plot(kind='scatter', x='temp', y='total_rentals', alpha=0.2);

We can visualise the distribution of some features, there maybe outliers that we need to remove in order for the model not trying to fit through these points

In [ ]:
bikes.hist(bins=50, figsize=(20,15));

From the above linear regression equation, we want to find a set of coefficients that best predict the total rentals. The features in our input are temperature, windspeed, holiday etc...
An example would be something like this:

$total\_rentals = 20 + -2 \cdot temp + -3 \cdot windspeed\ +\ ...\ +\ 0.1 \cdot humidity$

#### Let's build a Linear Regression Model in sklearn to apply to the bikeshare dataset
- Select the appropriate features to give to the model as inputs
- Perform feature engineering to transform the data types into the right format to be useful for the model to learn
- Compare train and test error. Eventhough we mentioned in the introduction that the best set of coefficients is the one that minimises the train MSE, test MSE is actually more important in evaluating the performance of an ML model. It shows you how your model is going to perform on new unseen data which is eventually what it needs to perform in reality. Therefore we are going to check on both the train error, as well as the test error for every modification steps and for new model selection.

In [ ]:
# Import LinearRegression model from sklearn
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

import numpy as np

## We need to define a train function to reduce the amount of typing
def train_linear(features, label, test_size=0.2):
    """
    Ins: features, labels, test_size
    Outs: None but print out train MSE, test MSE
    """
    X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=test_size, random_state=42)
    # Instantiate a linear regression model and fit it through our training data
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    # Make predictions on our training
    training_preds = linreg.predict(X_train)
    
    # Calculate and print the MSE
    print('Training RMSE:', np.sqrt(metrics.mean_squared_error(y_train, training_preds)))
    # Now apply the trained model on our test data and calculate the test error
    test_preds = linreg.predict(X_test)
    print('Test RMSE:', np.sqrt(metrics.mean_squared_error(y_test, test_preds)))

Create the feature vector X and the target y, for now just select all the features and in its current form


Call the function `train_linear` to train the initiated model

#### Exercise 1:
- Try out different set of features and report the test RMSE 
- You can try with different subset of features and see how they reduce the RMSE

For continuous variable prediction, accuracy is not a good metrics, since we can never have the predicted the value to match 100% with the target value (remember the noise?). Therefore, we need to look at the errors that the prediction has and try to reduce it.
Looking at a single test MSE or train MSE is not very informative as they are relative errors which describes the difference between the predicted outputs and the actual outputs. We normally want to look at the trend of these errors as trying out different models or after certain modifications to the features, then select the model or the configuration that yields the lowest train and test error. More on this later. For now, there are two important things that we want you to have in mind:
- Know how to create a train function to take in input features and create a model to predict the output labels
- The current test MSE and train MSE are approximately 156.006 and 154.618 respectively when you give the model all the input features without any modification or preprocessing steps

Let's use this as a baseline and we are going to see how other models and feature engineering techniques that you are going to learn later perform against this naive linear regression model.

### Feature Engineering

As we see from the above results, the columns `season`,`holiday`,`workingday` and `weather` should be of `categorical` data type.But the current data type is `integer` for those columns. Let us transform the dataset in the following ways so we can make it more useful for the model to learn.

- Extract more information from the current datetime by creating new columns `date`, `hour`, `weekDay`, `month` from `datetime` column.
- Convert the datatype of `season`, `holiday`, `workingday` and `weather` to category.
- Drop the datetime column as we already extracted useful features from it.
- Drop the `atemp` column because it conveys the same information as the `temp` column

In [ ]:
# Extract date, year, weekday and month into separate columns



In [ ]:
# Classify our features into categorical and numerical, at the same time we will drop some of the columns 


Voila, with just a few simple pre-processing steps to engineer the feature vectors, we are able to reduce both the train and the test RMSE, which is a significant amount of improvement. 
Let's learn one more techniques to handle categorical variable called `one-hot encoding`.

### One hot encoding ###

Categorical data are represented using integers and hence some ordinal relationships may be wrongly interpreted by the learning algorithms.
For example if there are five categories 'A', 'B', 'C', 'D' and 'E' that do not have any ordinal meanings, interprete them using integers from 1 to 5 may accidentially introduce comparability. Therefore, one-hot encoding can represent each of these categories using 5 binary vectors,or to be useful to the ML model, 4 of them are going to be used as the 5th vector can be infered from the first four. This is to remove collinearity between input features. Pandas has an option to remove this redundant column.

For example, the season column which contains 4 integers: 1 - Spring, 2 - Summer, 3 - Autumn and 4- Winter can be represented as 4 binary vectors:
- 1: [1, 0, 0, 0]
- 2: [0, 1, 0, 0]
- 3: [0, 0, 1, 0]
- 4: [0, 0, 0, 1]

In [ ]:
#bikes.season: perform comparsion between season 1 and season 4, which should not have order
bikes['season'][1] < bikes['season'][10884]

In [ ]:
# pandas get_dummies convert the categorical season column into 4 binary vectors representation, drop_first option if set to True to remove the first column,
# leave 3 of them for the input to the model


Eventhough the test MSE after one-hot encoding is not improved much compared to previous feature engineering steps, however just one simple function call to encode the categorical variable already worths a few percent improvement to the test error which is equally satisfying. 

## Pros/ Cons of Linear Regression
Advantages of linear regression:

- Simple to explain.
- Highly interpretable.
- Model training and prediction are fast.

Disadvantages of linear regression:

- Presumes a linear relationship between the features and the response.
- Performance is in general not competitive with other ML models.


#### (Optional) Feel free to play around with the features to see how they can help reducing the RMSE

#### Polynomial Features
- order 2 is ok to run
- order 4 is taking forever to run!!!
- How about polynomial features with regularisation?